In [1]:
import pandas as pd
u = pd.read_csv("U.csv")

In [2]:
def get_categories(df: pd.DataFrame, columns: list[str] = ["CLASS_" + str(x) for x in range(1,5)]):
    """Given a dataframe and list of columns, returns a dictionary with the given columns as keys and the unique elements of the columns (as a list)
    as the value. This is primarily to gather an iterable collection to be used in generating a drop down selection for the user given any universe."""
    result = {}
    for cat in columns:
        result[cat] = list(df[cat].unique())
    return result

#get_categories(u)
get_categories(u, ["RATING"])

{'RATING': ['AAA', 'AA', 'A', 'BBB', 'NA_character_', 'BB', 'NR', 'B']}

In [3]:
# def filter(df: pd.DataFrame, classifications: list[str], rating: str, dur_cell):
#     """filters = [df["CLASS_"+str(x)] == classifications[x] for x in classifications.keys()].append(df["RATING"] == rating)
#     for filter in filters:
#         df = df[filter]""" #Scaleable, but slow compared to brute force filter
#     df = df[(df["CLASS_1"] == classifications[1]) & (df["CLASS_2"] == classifications[2]) & (df["CLASS_3"] == classifications[3]) & (df["CLASS_4"] == classifications[4]) & (df["RATING"] == rating)]
#     return df



def filter(
    df: pd.DataFrame,
    classifications: dict,
    rating: str,
    dur_cell_min: int,
    dur_cell_max: int,
):
    """Given a dataframe, adictionary of classification, rating, and dur_cell, returns a universe filtered by the
    constraints derived from the parameters. That is, a universe such that each bond fills each parameters given

    :param df: an unindexed dataframe of the bond universe U
    :param dict classification: a dictionary with integer keys corresponding to each classification field, and
    string values of the classifcation
    :param str rating: a bond rating, ranging from BBB to AAA
    :param int dur_cell_min: Floor EFFDUR (years)
    :param int dur_cell_max: Ceiling EFFDUR (years)

    :return: a filtered dataframe"""

    # filters = [df["CLASS_"+str(x)] == classifications[x] for x in classifications.keys()].append(df["RATING"] == rating)
    # for filter in filters:
    #    df = df[filter]"""     # Scaleable, but slow compared to brute force filter

    df = df[
        (df["CLASS_1"] == classifications[1])
        & (df["CLASS_2"] == classifications[2])
        & (df["CLASS_3"] == classifications[3])
        & (df["CLASS_4"] == classifications[4])
        & (df["RATING"] == rating)
        & (df["EFFDUR"] > dur_cell_min)
        & (df["EFFDUR"] < dur_cell_max)
    ]
    return df

    
def summarize(df: pd.DataFrame, classifications: list[str], rating: str, dur_cell_min, dur_cell_max):
    df = filter(df, classifications, rating, dur_cell_min, dur_cell_max)
    result = {"OAS": {}, "YTM": {}}
    result["mv"] = df["MV"].sum()
    for metric in ["OAS", "YTM"]:
        result[metric]["min"] = df[metric].min()
        result[metric]["max"] = df[metric].max()
        result[metric]["mean"] = df[metric].mean()
        result[metric]["median"] = df[metric].median()
    return result


filter(u, {1: "CORPORATES", 2: "INDUSTRIAL", 3: "TECHNOLOGY", 4: "TECHNOLOGY"}, "AAA", 3, 17)
summarize(u, {1: "CORPORATES", 2: "INDUSTRIAL", 3: "TECHNOLOGY", 4: "TECHNOLOGY"}, "AAA", 3, 5)

{'OAS': {'min': 2.0, 'max': 7.0, 'mean': 4.666666666666667, 'median': 5.0},
 'YTM': {'min': 3.83458, 'max': 3.95094, 'mean': 3.88071, 'median': 3.85661},
 'mv': 11598872.04}

In [4]:
def generate_class_weights(df: pd.DataFrame, class_column: str="CLASS_2"):
    for field in df[class_column].unique():
        df[field] = (df[class_column]==field).apply(lambda x: 1 if x else 0)
    return df

u = generate_class_weights(u)

In [5]:

from pulp import *

def optimize(
    u: pd.DataFrame,
    target: str = "OAS",
    date: pd.Timestamp = pd.Timestamp("2023-03-31"),
    sec_weight: float = 0.05,
    max_indiv_weight: float = 0.02,
    delta: int = 3,
):
    # Initialize the Problem
    lp = LpProblem("bond-portfo-optimizer", LpMaximize)

    # Per Jorge, the portfolio is to be evaluated for only 1 of the two effective dates
    # Should this not be the case, we'd need to uncomment line 20, and remove the filter in line 21
    u = u[u["EFFDATE"].apply(pd.Timestamp) == date]

    # This seems unnecessary given our indexed dataframe; however, we need to ensure no negative values
    # are recorded for a bond; that is, avoid any optimal solution which would involve taking a short
    # position. This is a notable danger, as a solution with short positions exist with negative
    # values to 'limit exposure' to certain sectors (i.e reduce net weight in Industrials under Class 2)
    bond_index = LpVariable.dicts(
        "Bond_ID", list(u["SECURITY_ALIAS"]), lowBound=0, upBound=max_indiv_weight, cat="Continuous"
    )
    print(bond_index)

    # objective function declaration
    obj = dict(zip(u["SECURITY_ALIAS"], u[target]))
    lp += lpSum([obj[b] * bond_index[b] for b in bond_index.keys()]), "Total weighted OAS/YTM"

    # restrict our weights to 100% total
    lp += lpSum([1 * bond_index[b] for b in bond_index.keys()]) <= 1, "Maximum total weight"

    # Add restrictions for the 3 categories under class 2
    for cat in u.columns[-3:]:
        category = dict(zip(u["SECURITY_ALIAS"], u[cat]))
        
        lp += (
            lpSum([category[b] * bond_index[b] for b in bond_index.keys()])
            <= sec_weight, cat+" Ceiling"
        )
    
    
    total_dur = u["EFFDUR"].sum()
    eff_dur = dict(zip(u["SECURITY_ALIAS"], u["EFFDUR"].astype(float)))
    lp += lpSum([eff_dur[b] * bond_index[b] for b in bond_index.keys()]) <= (total_dur + delta), "Portfolio Duration + Delta"
    lp += lpSum([eff_dur[b] * bond_index[b] for b in bond_index.keys()]) >= (total_dur - delta), "Minus Delta"

    lp.solve()
    print(lp)
    return lp
optimize(u)
#for v in optimize(u).variables():
#    if v.varValue>0:
#        print(v.name, "=", v.varValue)

{533765: Bond_ID_533765, 5639014: Bond_ID_5639014, 5639015: Bond_ID_5639015, 5639016: Bond_ID_5639016, 5639090: Bond_ID_5639090, 5639022: Bond_ID_5639022, 5638940: Bond_ID_5638940, 5639000: Bond_ID_5639000, 5639001: Bond_ID_5639001, 5639136: Bond_ID_5639136, 5639013: Bond_ID_5639013, 5638751: Bond_ID_5638751, 5639025: Bond_ID_5639025, 5639041: Bond_ID_5639041, 5639042: Bond_ID_5639042, 5639089: Bond_ID_5639089, 5639026: Bond_ID_5639026, 5639027: Bond_ID_5639027, 5639028: Bond_ID_5639028, 5639094: Bond_ID_5639094, 5639095: Bond_ID_5639095, 5639107: Bond_ID_5639107, 5639194: Bond_ID_5639194, 5639192: Bond_ID_5639192, 5639080: Bond_ID_5639080, 5639188: Bond_ID_5639188, 5639052: Bond_ID_5639052, 5638610: Bond_ID_5638610, 5639065: Bond_ID_5639065, 5639071: Bond_ID_5639071, 5639072: Bond_ID_5639072, 5639190: Bond_ID_5639190, 5638986: Bond_ID_5638986, 5639002: Bond_ID_5639002, 5639112: Bond_ID_5639112, 5639113: Bond_ID_5639113, 5639069: Bond_ID_5639069, 5639070: Bond_ID_5639070, 5639039: Bond

bond-portfo-optimizer:
MAXIMIZE
170.0*Bond_ID_1001884 + 334.0*Bond_ID_1099903 + 315.0*Bond_ID_1099969 + 275.0*Bond_ID_1100047 + 93.0*Bond_ID_1100103 + 116.0*Bond_ID_1100109 + 42.0*Bond_ID_1100111 + 29.0*Bond_ID_1100112 + 207.0*Bond_ID_1100113 + 82.0*Bond_ID_1100119 + 152.0*Bond_ID_1100120 + 100.0*Bond_ID_1100121 + 236.0*Bond_ID_1100124 + 136.0*Bond_ID_1100142 + 79.0*Bond_ID_1100143 + 68.0*Bond_ID_1100144 + 196.0*Bond_ID_1100145 + 41.0*Bond_ID_1100148 + 74.0*Bond_ID_1100150 + 83.0*Bond_ID_1100157 + 162.0*Bond_ID_1100160 + 290.0*Bond_ID_1100169 + 83.0*Bond_ID_1100172 + 237.0*Bond_ID_1100187 + 71.0*Bond_ID_1100198 + 129.0*Bond_ID_1100200 + 207.0*Bond_ID_1100201 + 222.0*Bond_ID_1100202 + 64.0*Bond_ID_1100203 + 220.0*Bond_ID_1100204 + 550.0*Bond_ID_1100206 + 664.0*Bond_ID_1100207 + 167.0*Bond_ID_1100212 + 154.0*Bond_ID_1100227 + 165.0*Bond_ID_1100228 + 82.0*Bond_ID_1100230 + 140.0*Bond_ID_1100234 + 331.0*Bond_ID_1100259 + 347.0*Bond_ID_1100260 + 138.0*Bond_ID_1100268 + 115.0*Bond_ID_1100297